In [ ]:
# Transformers installation
!pip install transformers datasets

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cuda"
model_id = "mistralai/Mistral-7B-v0.1"
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
encodings = tokenizer("The quick brown fox", return_tensors="pt").to(device)

In [ ]:
# Define the input text X and the corresponding output text Y
input_text = "What is the capital of France?"
output_text = "The capital of France is Paris."

# Tokenize the input and output texts
input_ids = tokenizer.encode(input_text, return_tensors="pt")
output_ids = tokenizer.encode(output_text, return_tensors="pt")

# Concatenate the input and output token IDs TODO test
input_output_ids = torch.cat((input_ids, output_ids), dim=-1)
trg_len = len(output_ids)
target_ids = input_output_ids.clone()
target_ids[:, :-trg_len] = -100  #wtf does this do

In [ ]:
def calculatePerplexity():
    max_length = 1024
    seq_len = encodings.input_ids.size(1)
    
    nlls = []
    end_loc = min(max_length, seq_len)
    trg_len = end_loc # may be different from stride on last loop
    input_ids = encodings.input_ids[:, 0:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100  #wtf does this do
    
    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)
    
        # loss is calculated using CrossEntropyLoss which averages over valid labels
        # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
        # to the left by 1.
        neg_log_likelihood = outputs.loss
    
    nlls.append(neg_log_likelihood)    
    
    ppl = torch.exp(torch.stack(nlls).mean())
    print(neg_log_likelihood, ppl)

#Mass Calculation

In [7]:
def calculatePerplexity1(input, output):
    # Your perplexity calculation logic here
    # Replace this with your actual implementation
    return 3

import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('prompts_completions.csv')
output_col = 'claude'

# Extract the prompts and outputs arrays from the DataFrame
prompts = df['prompt'].tolist()
outputs = df[output_col].tolist()

# Create a new column in the DataFrame to store the perplexity values
df['perplexity'] = ''

# Iterate over the DataFrame rows and calculate perplexity for each prompt-output pair
for index, row in df.iterrows():
    prompt = row['prompt']
    output = row[output_col]
    perplexity = calculatePerplexity1(prompt, output)
    df.at[index, 'perplexity'] = perplexity

# Save the updated DataFrame back to a CSV file
df.to_csv('prompts_completions1.csv', index=False)
print(df)

                                               prompt  \
0                   What is a man? A miserable little   
1                                    Once upon a time   
2                                    In a world where   
3     The sun was setting over the horizon, casting a   
4      She looked at her reflection in the mirror and   
5   The old man sat on the bench, feeding the pige...   
6   The spaceship landed on the alien planet, and ...   
7   The detective examined the crime scene, lookin...   
8   The ancient ruins held a secret that had been ...   
9   The robot's artificial intelligence had evolve...   
10  The young wizard held the wand tightly, knowin...   
11  The time traveler stepped out of the machine a...   
12  The artist stood back and admired their latest...   
13  The king sat on his throne, contemplating the ...   
14  The scientist made a groundbreaking discovery ...   
15  The soldier looked out over the battlefield, k...   
16  The dragon spread its wings